In [54]:
print('Materialisation Data Test')

Materialisation Data Test


In [55]:
import os
import compas
from compas.datastructures import Mesh, mesh_bounding_box_xy
from compas.geometry import Vector, Frame, Scale

HERE = os.getcwd()

FILE_I = os.path.join(HERE, 'blocks and ribs_RHINO', 'sessions', 'bm_vertical_equilibrium', 'simple_tripod.rv2')
FILE_O1 = os.path.join(HERE, 'blocks and ribs_RHINO', 'data', 'form.json')
FILE_O2 = os.path.join(HERE, 'blocks and ribs_RHINO', 'data', 'scaled_form.json')

session = compas.json_load(FILE_I)

mesh = Mesh.from_data(session['data']['form'])
loader_mesh = Mesh.from_data(session['data']['form'])

### to delete extra faces(more than 4 edges) if subdivided with catmulclark or other weird subdivision that connects the mesh with the ground

In [56]:
delete_faces =[]

for fkey in mesh.faces():
    if len(mesh.face_vertices(fkey)) > 4:
        delete_faces.append(fkey)

for fkey in delete_faces:       
    mesh.delete_face(fkey)
    mesh.remove_unused_vertices()

### scale up the form if needed

In [57]:
scaled_mesh = mesh.copy()

box_points = mesh_bounding_box_xy(scaled_mesh)
base_mesh = scaled_mesh.from_points(box_points)
centroid = base_mesh.centroid()
#print (centroid)
frame = Frame(centroid,Vector(1,0,0),Vector(0,1,0))

S = Scale.from_factors([100, 100, 100], frame)
scaled_mesh.transform(S)

### Visualise and export Initial Mesh

In [58]:
mesh.to_json(FILE_O1)
scaled_mesh.to_json(FILE_O2)

print(mesh)

<Mesh with 37 vertices, 24 faces, 60 edges>


In [59]:
from pythreejs import *
import numpy as np
from IPython.display import display

mesh = scaled_mesh.copy()

vertices = []
for face in mesh.faces():
    for v in mesh.face_vertices(face):
        xyz = mesh.vertex_attributes(v, "xyz")
        vertices.append(xyz)

# print(vertices)

In [60]:
vertices = BufferAttribute(
    array = np.array(vertices,dtype=np.float32),
    normalized = False)

# print(vertices)


In [63]:
geometry = BufferGeometry(
    attributes={'position': vertices})

print(geometry)

BufferGeometry(attributes={'position': <BufferAttribute shape=(96, 3), dtype=float32>})


In [64]:
geometry.exec_three_obj_method('computeVertexNormals')

mesh_3j = Mesh(geometry=geometry,
           material=MeshLambertMaterial(color='red'),
           position=[0,0,0])

In [65]:
print(geometry)

BufferGeometry(attributes={'position': <BufferAttribute shape=(96, 3), dtype=float32>})


In [66]:
print(type(mesh_3j))

<class 'pythreejs.objects.Mesh_autogen.Mesh'>


In [67]:
c = PerspectiveCamera(position=[0, 5, 5], up=[0, 1, 0],
                      children=[DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)])

scene=Scene(children=[mesh_3j,c, AmbientLight(color='#777777')])


renderer = Renderer(camera=c, 
                    scene=scene,
                    width=800,
                    height=800,
                    controls=[OrbitControls(controlling=c)])

/opt/miniconda3/envs/csd2/lib/python3.9/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


In [68]:
display(renderer)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, matrixWorldNeedsUpd…